In [3]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap

# 지정한 경로 가져옴
model_path = "/content/drive/MyDrive/DILAB/llama3-8b"

# 지정한 경로에서 토크나이저, Model들을 불러온다
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map = "auto", # GPU 자동 할당
    torch_dtype = "auto" # fp16 등 자동 감지
)

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",     # GPU 할당
)

# 테스트 프롬프트
# <<Prompt Formatting>>
  # 방법 1.
# prompt = "[INST] 삼성전자 신입사원 채용 우대조건은? \n[/INST]"

  # 방법 2.
messages = [{"role":"user", "content":"삼성전자 백엔드 파트 채용 우대사항은?"}]
prompt = tokenizer.apply_chat_template(
    messages,
	  tokenize=False,
	  add_generation_prompt=True # assistant의 답변 시작 위치까지 생성
)

# 텍스트 생성
output = pipe(
    prompt,
    max_new_tokens=500,     # 생성할 최대 토큰 수
    do_sample=True,        # 샘플링 방식 사용 (더 자연스럽게)
    temperature=0.7,       # 창의성 조절 (낮을수록 보수적)
    top_p=0.9,              # nucleus sampling
    eos_token_id=tokenizer.eos_token_id  # 문장이 끝났음을 뜻하는 eos_token_id를 명시하여, 모델이 끝을 알게 하기
)

# 출력 결과 확인
output_text = output[0]["generated_text"]

wrapped_text = textwrap.fill(output_text, width=70)

print(wrapped_text)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


<|begin_of_text|><|start_header_id|>user<|end_header_id|>  삼성전자 백엔드 파트
채용 우대사항은?<|eot_id|><|start_header_id|>assistant<|end_header_id|>  삼성전자
시스템 LSI사업부 인사팀


In [14]:
import os
print("현재 작업 디렉토리:", os.getcwd())


현재 작업 디렉토리: /content


In [10]:
!nvidia-smi

Wed Jul 23 09:20:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             91W /  400W |   31321MiB /  40960MiB |     46%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----